# Numerical Solution of the Two-Dimensional Helmholtz Equation using the Finite Element Method

This notebook illustrates the numerical solution of the two-dimensional wave equation for an harmonic excitation using the so called [Finite Element Method](https://en.wikipedia.org/wiki/Finite_element_method) (FEM).

## Problem Statement

The inhomogeneous linear [wave equation](https://en.wikipedia.org/wiki/Wave_equation) is given as

\begin{equation}
\Delta p(\mathbf{x}, t) - \frac{1}{c^2} \frac{\partial^2}{\partial t^2} p(\mathbf{x}, t) = - q(\mathbf{x}, t) ,
\end{equation}

where $p(\mathbf{x}, t)$ denotes the sound pressure at position $\mathbf{x}$, $c$ the speed of sound and $q(\mathbf{x}, t)$ the inhomogeneity.
For an harmonic excitation $q(\mathbf{x}, t) = \Re \{ Q(\mathbf{x}, \omega) \mathrm{e}^{\mathrm{j} \omega t} \}$ with frequency $\omega = 2 \pi f$ we choose the Ansatz $p(\mathbf{x}, t) = \Re \{ P(\mathbf{x}, \omega) \mathrm{e}^{\mathrm{j} \omega t} \}$ for the sound pressure.
Introduction of the complex quantities into the wave equation yields

\begin{equation}
\Delta P(\mathbf{x}, \omega) \mathrm{e}^{\mathrm{j} \omega t} + \frac{\omega^2}{c^2} P(\mathbf{x}, \omega) \mathrm{e}^{\mathrm{j} \omega t} = - Q(\mathbf{x}, \omega) \mathrm{e}^{\mathrm{j} \omega t} ,
\end{equation}

canceling out the $\mathrm{e}^{\mathrm{j} \omega t}$ terms yields the [Helmholtz equation](https://en.wikipedia.org/wiki/Helmholtz_equation)

\begin{equation}
\Delta P(\mathbf{x}, \omega) + \frac{\omega^2}{c^2} P(\mathbf{x}, \omega) = - Q(\mathbf{x}, \omega) .
\end{equation}

We aim for a numerical solution of the Helmholtz equation on the domain $V$ with respect to the homogeneous Dirichlet boundary condition

\begin{equation}
P(\mathbf{x}, \omega) = 0 \qquad \text{for } x \in \partial V 
\end{equation}

or the homogeneous Neumann boundary condition

\begin{equation}
\frac{\partial}{\partial n} P(\mathbf{x}, \omega) = 0 \qquad \text{for } x \in \partial V ,
\end{equation}

where $\partial V $ denotes the boundary of $V$.

## Variational Formulation

The FEM is based on expressing the partial differential equation (PDE) to be solved in its [variational](https://en.wikipedia.org/wiki/Calculus_of_variations) or weak form.
The first step towards this formulation is to multiply the Helmholtz equation by the test function $V(\mathbf{x}, \omega)$

\begin{equation}
\Delta P(\mathbf{x}, \omega) \cdot V(\mathbf{x}, \omega)  + \frac{\omega^2}{c^2} P(\mathbf{x}, \omega) \cdot V(\mathbf{x}, \omega) = - Q(\mathbf{x}, \omega) \cdot V(\mathbf{x}, \omega) ,
\end{equation}

followed by integration over the domain $V$

\begin{equation}
\int_V \Delta P(\mathbf{x}, \omega) \cdot V(\mathbf{x}, \omega) \mathrm{d}x  + \frac{\omega^2}{c^2} \int_V P(\mathbf{x}, \omega) \cdot V(\mathbf{x}, \omega) \mathrm{d}x = - \int_V Q(\mathbf{x}, \omega) \cdot V(\mathbf{x}, \omega) \mathrm{d}x ,
\end{equation}

where $\mathrm{d}x$ denotes a suitably chosen differential element for integration.
Application of [Green's first identity](https://en.wikipedia.org/wiki/Green%27s_identities) yields

\begin{equation}
- \int_V \nabla P(\mathbf{x}, \omega) \cdot \nabla V(\mathbf{x}, \omega) \mathrm{d}x  + 
\int_{\partial V} V(\mathbf{x}, \omega) \frac{\partial}{\partial n}  P(\mathbf{x}, \omega) \mathrm{d}s
+ \frac{\omega^2}{c^2} \int_V P(\mathbf{x}, \omega) V(\mathbf{x}, \omega) \mathrm{d}x = 
- \int_V Q(\mathbf{x}, \omega) V(\mathbf{x}, \omega) \mathrm{d}x .
\end{equation}

This way the differential order of the first integral is lowered which is advisable for application of the FEM.
The second integral vanishes as the variation formulation requires $V(\mathbf{x}, \omega) = 0$ on $\partial V$ where $P(\mathbf{x}, \omega)$ is known by - for instance fixed - boundary conditions.
This results in the variational/weak formulation of the Helmholtz equation

\begin{equation}
- \int_V \nabla P(\mathbf{x}, \omega) \cdot \nabla V(\mathbf{x}, \omega) \mathrm{d}x  
+ \frac{\omega^2}{c^2} \int_V P(\mathbf{x}, \omega) V(\mathbf{x}, \omega) \mathrm{d}x = 
- \int_V Q(\mathbf{x}, \omega) V(\mathbf{x}, \omega) \mathrm{d}x
\end{equation}

It is common to express this in terms of the bilinear $a(P, V)$ and linear $L(V)$ forms 

\begin{equation}
a(P, V) = \frac{\omega^2}{c^2} \int_V P(\mathbf{x}, \omega) V(\mathbf{x}, \omega) \mathrm{d}x - \int_V \nabla P(\mathbf{x}, \omega) \cdot \nabla V(\mathbf{x}, \omega) \mathrm{d}x ,
\end{equation}

\begin{equation}
L(V) = - \int_V Q(\mathbf{x}, \omega) V(\mathbf{x}, \omega) \mathrm{d}x ,
\end{equation}

where obviously

\begin{equation}
a(P, V) = L(V) .
\end{equation}

## Numerical Solution

The numerical solution of the variational problem is based on [FEniCS](https://fenicsproject.org/), an open-source framework for numerical solution of PDEs.
Its high-level Python interface `dolfin` is used in the following to define the problem and compute its solution.
The implementation is based on the variational formulation derived above.
It is common in the FEM to denote the solution of the problem by $u$ and the test function by $v$.
The definition of the problem in FEniCS is very close to the mathematical formulation of the problem.
We limit ourselves to real-valued $P(\mathbf{x}, \omega)$ due to the assumption of Dirichlet or Neumann boundary conditions.

For the subsequent examples the solution of the inhomogeneous wave equation for a point source $Q(\mathbf{x}) = \delta(\mathbf{x}-\mathbf{x_s})$ at position $\mathbf{x_s}$ is computed using the FEM.
A function is defined for this purpose, as well as for the plotting of the resulting sound field.

In [ ]:
from dolfin import *
import mshr
import matplotlib.pyplot as plt
%matplotlib inline


def Helmholtz_2D(mesh, frequency, xs, neumann_bc=True, c=343):
    '''Numerical solution of the two-dimensional Helmholtz equation 
       using the FEM'''
    
    # squared wavenumber
    k2 = (Constant(2*pi*frequency)/Constant(c))**2
    
    # define function space
    V = FunctionSpace(mesh, "CG", 1)

    # define boundary conditions
    if neumann_bc:
        bcs = None
    else:
        bcs = DirichletBC(V, Constant(0.), "on_boundary")
    
    # define variational problem
    u = TrialFunction(V)
    v = TestFunction(V)

    a = k2 * inner(u, v) * dx - inner(nabla_grad(u), nabla_grad(v)) * dx
    L = Constant(0.0) * v * dx
    
    A, b = assemble_system(a, L, bcs)
    
    # define inhomogenity
    delta = PointSource(V, xs, -1)  # account for negative sign in inhomogeneity
    delta.apply(b)

    # compute solution
    u = Function(V)
    solve(A, u.vector(), b)

    return u


def plot_soundfield(u):
    '''plots solution of FEM-based simulation'''
    fig = plt.figure(figsize=(10,10))
    fig = plot(u)
    plt.title(r'$P(\mathbf{x}, \omega)$')
    plt.xlabel(r'$x$ / m')
    plt.ylabel(r'$y$ / m')
    plt.colorbar(fig, fraction=0.038, pad=0.04);

### Validation with Kundt's Tube

The two-dimensional sound field in a rectangular room with rigid (Neumann) boundaries is computed for a point source located very near the left boundary ($x\approx 0$) and centered ($y=0$).
The room is larger in x-dimension and much smaller in y-dimension compared to the chosen wavelength.
By using these values, the simulation geometry is identical to [Kundt's tube](https://en.wikipedia.org/wiki/Kundt%27s_tube) (cf. e.g. chapter 6 in the textbook [Möser, M. (2009): Engineering Acoustics, Springer, Berlin, 2nd ed.](https://doi.org/10.1007/978-3-540-92723-5)) and the result can be compared to its known analytic solution, i.e. a standing wave phenomenon.
Hence, we can validate our simulation environment, by checking for

1. positive pressure amplitude at the left and right boundary in the first plot for showing the amplitude of the pressure at time instance $t=0$ and
2. characteristic standing wave node pattern for rigid (Neumann) boundary conditions in the second plot. Here maxima occur at $x=\mu \frac{\lambda}{2}$ with $\mu=0,1,2,...$ up to the length of the tube. Note that this plot depicts the magnitude of pressure.

In [ ]:
lmb = 1  # wavelength in m
c = 343  # speed of sound in m/s
# define geometry and mesh
mesh = RectangleMesh(Point(0, -lmb/10), Point(5*lmb, +lmb/10), 200, 200, "right/left")

# compute solution
u = Helmholtz_2D(mesh, c/lmb, Point(lmb/100,0))

# plot sound field
plot_soundfield(u)
plot_soundfield(abs(u))
plt.title(r'$|P(\mathbf{x}, \omega)|$');

### Sound Field in a Rectangular Room with Sound-Hard Boundaries

Now, after validation, we can use the same code as above for a more sophisticated example.
The two-dimensional sound field in a rectangular room whose height is very small compared to the wavelength and with rigid (Neumann) boundaries is computed for the frequency $f=1000$ Hz and source position $x_s = (1.2,3.2)$ m. The phenomena look more complex, it is however still only due to interference of direct and reflected waves.

In [ ]:
# define geometry and mesh
mesh = RectangleMesh(Point(0,0), Point(5,4), 200, 200, "right/left")

# compute solution
u = Helmholtz_2D(mesh, 1000, Point(1.2,3.2))

# plot sound field
plot_soundfield(u)
plot_soundfield(abs(u))
plt.title(r'$|P(\mathbf{x}, \omega)|$');

### Sound Field in Two Coupled Rectangular Rooms

The two-dimensional sound field in two coupled rectangular rooms with sound-hard boundaries (Neumann boundary conditions) is computed for the frequency $f=1000$ Hz and source position $x_s = (2,0.5)$ m. First the geometry is defined and plotted. Note the mesh is generated with a low number of elements for ease of illustration. A higher resultion is used for the simulations later.

In [ ]:
# define geometry and mesh
domain = mshr.Rectangle(Point(0, 0), Point(3,4)) + mshr.Rectangle(Point(3, 1.5), Point(3.5, 2.5)) + mshr.Rectangle(Point(3.5, 0), Point(6, 4))
mesh2 = mshr.generate_mesh(domain, 20)

plot(mesh2);

In [ ]:
# compute solution
mesh2 = mshr.generate_mesh(domain, 100)
u = Helmholtz_2D(mesh2, 1000, Point(2, .5))

# plot sound field
plot_soundfield(u)
plot_soundfield(abs(u))
plt.title(r'$|P(\mathbf{x}, \omega)|$');

**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).